In [114]:
from optimization.koneksi import ConDB
import random
import math
import copy
import time
import json
import datetime
import numpy as np

class GA_VRP(object):
    def __init__(self,population_size = 50,crossover_rate=0.8,mutation_rate=0.2,max_iter = 300,max_idem = 15,random_state=None):
        self.db = ConDB()
        
        # parameter setting
        self.population_size = population_size #max size of population
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.max_iter = max_iter #max iteration of tabu search
        self.max_idem = max_idem #stop if the best fitness doesn't increase for max_idem iteration

        # set initial solution
        self.init_solution = [] #2D list of nodes, [[node1,node2,....],[node4,node5,....]]
        self.rest_nodes = [] #1D list of nodes, [node1,node2,node4,node5,....]

        # data model setting
        self.tour = None #POI yang dipilih oleh user untuk dikunjungi
        self.hotel = None #hotel yang dipilih oleh user
        self.timematrix = None
        self.max_travel_time = None
        self.travel_days = None
        
        #degree of interest (DOI for MAUT) setting
        self.degree_waktu = 1
        self.degree_tarif = 1
        self.degree_rating = 1
        self.degree_poi = 1
        self.degree_poi_penalty = 1
        self.degree_time_penalty = 1
        
        #scaler setting
        self.min_rating = None
        self.max_rating = None
        self.min_tarif = None
        self.max_tarif = None
        self.min_waktu = None
        self.max_waktu = None
        self.min_poi = None
        self.max_poi = None
        self.min_poi_penalty = None
        self.max_poi_penalty = None
        self.min_time_penalty = None
        self.max_time_penalty = None
        
        #set random seed
        if random_state != None:
            random.seed(random_state)
    
    def set_model(self,tour,hotel,timematrix,init_solution=[],travel_days = 3, depart_time = datetime.time(8,0,0),max_travel_time = datetime.time(20,0,0),degree_waktu = 1,degree_tarif = 1,degree_rating = 1):
        #initiate model
        self.tour = copy.deepcopy(tour)
        self.hotel = copy.deepcopy(hotel)
        self.travel_days = travel_days
        self.hotel.depart_time = depart_time
        self.max_travel_time = max_travel_time
        self.timematrix = copy.deepcopy(timematrix)
        
        self.degree_waktu = degree_waktu
        self.degree_tarif = degree_tarif
        self.degree_rating = degree_rating
        
        self.min_rating = min([node.rating for node in self.tour])
        self.max_rating = max([node.rating for node in self.tour])
        self.min_tarif = min([node.tarif for node in self.tour])
        self.max_tarif = sum([node.tarif for node in self.tour])
        self.min_waktu = 0
        self.max_waktu = (self.diff_second_between_time(depart_time,max_travel_time))*self.travel_days
        self.min_poi = 0
        self.max_poi = len(self.tour)
        self.min_poi_penalty = 0
        self.max_poi_penalty = len(self.tour)
        self.min_time_penalty = 0
        self.max_time_penalty = ((24*3600)-self.diff_second_between_time(max_travel_time,depart_time))*travel_days
    
    def set_max_iter(self,max_iter):
        self.max_iter = max_iter
    
    def time_to_second(self,time):
        return (time.hour*3600)+(time.minute*60)+time.second
    
    def second_to_time(self,second):
        second = int(second)
        return datetime.time(second//3600,(second//60)%60,0) #ignore second detail
    
    def diff_second_between_time(self,time_a,time_b):
        #input: time_a and time_b, datetime.time()
        #output: time_b - time_a, seconds (int)
        return self.time_to_second(time_b) - self.time_to_second(time_a)
    
    def min_max_scaler(self,min_value,max_value,value):
        if max_value-min_value == 0:
            return 0
        else:
            return (value-min_value)/(max_value-min_value)
    
    def MAUT(self,solutions,consider_total_poi = True,use_penalty = True):
        #input: optimization solutions, format = [{"index":[],"waktu":[],"rating":[],"tarif":[]},...]
        #output: fitness value calculated using MAUT
        
        #concat all attribute lists (except for waktu)
        index_ls = sum([i['index'] for i in solutions],[])
        rating_ls = sum([i['rating'] for i in solutions],[])
        tarif_ls = sum([i['tarif'] for i in solutions],[])
        
        waktu_ls = [i['waktu'] for i in solutions]
        
        #rating
        avg_rating = sum(rating_ls)/len(rating_ls)
        score_rating = self.min_max_scaler(self.min_rating,self.max_rating,avg_rating)*self.degree_rating
        
        #tarif
        sum_tarif = sum(tarif_ls)
        score_tarif = (1-self.min_max_scaler(self.min_tarif,self.max_tarif,sum_tarif)) * self.degree_tarif
        
        #waktu
        waktu_per_day = [self.diff_second_between_time(i[0],i[-1]) for i in waktu_ls]
        sum_waktu = sum(waktu_per_day)
        score_waktu = (1-self.min_max_scaler(self.min_waktu,self.max_waktu,sum_waktu))*self.degree_waktu
        
        #poi
        count_poi = len(index_ls)
        score_poi = self.min_max_scaler(self.min_poi,self.max_poi,count_poi) if consider_total_poi == True else 0
        
        if use_penalty==True:
            #poi penalty
            penalty_index = [node._id for node in self.tour if node._id not in index_ls]
            count_penalty = len(penalty_index)
            score_poipenalty = (1-self.min_max_scaler(self.min_poi_penalty,self.max_poi_penalty,count_penalty)) * self.degree_poi_penalty
            
            #time penalty
            penalty_per_day = [max(self.diff_second_between_time(i[-1],self.max_travel_time),0) for i in waktu_ls]
            sum_time_penalty = sum(penalty_per_day)
            score_timepenalty = (1-self.min_max_scaler(self.min_time_penalty,self.max_time_penalty,sum_time_penalty)) * self.degree_time_penalty
        else:
            score_poipenalty = 0
            score_timepenalty = 0
            
        #MAUT
        degree_rating = self.degree_rating
        degree_tarif = self.degree_tarif
        degree_waktu = self.degree_waktu
        degree_poi = self.degree_poi if consider_total_poi == True else 0
        degree_poi_penalty = self.degree_poi_penalty if use_penalty == True else 0
        degree_time_penalty = self.degree_time_penalty if use_penalty == True else 0

        pembilang = score_rating+score_tarif+score_waktu+score_poi+score_poipenalty+score_timepenalty
        penyebut = degree_rating+degree_tarif+degree_waktu+degree_poi+degree_poi_penalty+degree_time_penalty
        maut = pembilang/penyebut
        return maut
    
    def MAUT_between_two_nodes(self,current_node,next_node):
        score_rating = self.degree_rating * self.min_max_scaler(self.min_rating,self.max_rating,next_node.rating)
        score_tarif = self.degree_tarif * (1-self.min_max_scaler(self.min_tarif,self.max_tarif,next_node.rating))
        score_waktu = self.degree_waktu * (1-self.min_max_scaler(self.min_waktu,self.max_waktu,self.timematrix[current_node._id][next_node._id]['waktu']))
        maut = (score_rating+score_tarif+score_waktu)/(self.degree_rating+self.degree_tarif+self.degree_waktu)
        return maut
    
    def next_node_check(self,current_node,next_node):
        time_needed = self.time_to_second(current_node.depart_time)+self.timematrix[current_node._id][next_node._id]["waktu"]+next_node.waktu_kunjungan
        time_limit = self.time_to_second(self.max_travel_time)
        if (time_needed <= time_limit) and (time_needed <= self.time_to_second(next_node.jam_tutup)):
            return True
        else:
            return False
    
    def set_next_node_depart_arrive_time(self,current_node,next_node):
        arrive_time = self.time_to_second(current_node.depart_time)+self.timematrix[current_node._id][next_node._id]["waktu"]
        arrive_time = max([arrive_time,self.time_to_second(next_node.jam_buka)])
        next_node.arrive_time = self.second_to_time(arrive_time)
        if next_node.tipe.lower() != "hotel":
            next_node.depart_time = self.second_to_time(arrive_time+next_node.waktu_kunjungan)
        return next_node
    
    def solution_list_of_nodes_to_dict(self, solutions):
        solution_dict = []
        for day in solutions:
            day_solution = {"index":[],"waktu":[self.hotel.depart_time],"rating":[],"tarif":[]}
            day_solution['index'] = [i._id for i in day]
            day_solution['rating'] = [i.rating for i in day]
            day_solution['tarif'] = [i.tarif for i in day]
            
            last_waktu = self.time_to_second(day[-1].depart_time) + self.timematrix[day[-1]._id][self.hotel._id]['waktu']
            day_solution['waktu'].extend([i.arrive_time for i in day] + [self.second_to_time(last_waktu)])
            
            solution_dict.append(day_solution)
        return solution_dict
    
    def split_itinerary(self,init_itinerary):
        final_solution = [] #2d list of nodes
        day = 1
        tabu_nodes = [] #list of tabu node's id
        while day <= self.travel_days:
            current_node = self.hotel
            day_solution = [] #list of nodes
            next_node_candidates = [node for node in init_itinerary if node._id not in tabu_nodes]
            for i in range(len(next_node_candidates)):
                if self.next_node_check(current_node,next_node_candidates[i]):
                    next_node_candidates[i] = self.set_next_node_depart_arrive_time(current_node,next_node_candidates[i])
                    day_solution.append(next_node_candidates[i])
                    tabu_nodes.append(next_node_candidates[i]._id)
                    current_node = next_node_candidates[i]
            
            if len(day_solution) > 0:
                final_solution.append(day_solution)
            
            if len(tabu_nodes) == len(self.tour):
                break
            
            day += 1
        return final_solution
    
    def partially_mapped_crossover(self,parent1,parent2): #PMX
        start,end = sorted(random.sample(range(len(parent1)), 2))
        child1 = [None] * len(parent1)
        child2 = copy.deepcopy(child1)
        mapping1 = {parent1[i]._id:parent2[i] for i in range(start,end+1)}
        mapping2 = {parent2[i]._id:parent1[i] for i in range(start,end+1)}

        for i in range(len(parent1)):
            if start <= i <= end:
                child1[i],child2[i] = parent1[i],parent2[i] 
            else:
                gene1,gene2 = parent2[i],parent1[i]
                while gene1._id in mapping1:
                    gene1 = mapping1[gene1._id]
                while gene2._id in mapping2:
                    gene2 = mapping2[gene2._id]
                child1[i],child2[i] = gene1,gene2

        return child1,child2
    
    def swap_mutation(self,individual):
        individu = copy.deepcopy(individual)
        pos1,pos2 = random.sample(range(len(individu)),2)
        individu[pos1],individu[pos2] = individu[pos2],individu[pos1]
        return individu
    
    def construct_solution(self):
        solution_dict = {}
        fitness = 0
        
        idem_counter = 0
        population = [random.sample(self.tour,len(self.tour)) for i in range(self.population_size)]
        for i in range(self.max_iter):
                        
            #crossover and mutation
            offspring = []
            for ind in range(0,self.population_size,2):
                parent1,parent2 = population[ind],population[ind+1]

                #crossover
                if random.uniform(0,1) < self.crossover_rate:
                    child1,child2 = self.partially_mapped_crossover(parent1,parent2)
                else:
                    child1,child2 = copy.deepcopy(parent1),copy.deepcopy(parent2)
                
                #mutation
                if random.uniform(0,1) < self.mutation_rate:
                    child1 = self.swap_mutation(child1)
                if random.uniform(0,1) < self.mutation_rate:
                    child2 = self.swap_mutation(child2)
                
                offspring.append(child1)
                offspring.append(child2)
            
            #selection roulette wheel
            all_individuals = population + offspring
            fitness_values = [self.MAUT(self.solution_list_of_nodes_to_dict(self.split_itinerary(individual))) for individual in all_individuals]
            
            population = random.choices(all_individuals,weights=np.array(fitness_values)/sum(fitness_values),k=self.population_size)

            best_solution = max(population,key=lambda x: self.MAUT(self.solution_list_of_nodes_to_dict(self.split_itinerary(x))))
            best_solution_dict = self.solution_list_of_nodes_to_dict(self.split_itinerary(best_solution))
            best_fitness = self.MAUT(best_solution_dict)
            
            if fitness < best_fitness:
                fitness = best_fitness
                solution_dict = best_solution_dict
                idem_counter = 0
            else:
                idem_counter += 1
                if idem_counter > self.max_idem:
                    return solution_dict,fitness
        
        return solution_dict,fitness

In [115]:
from optimization.koneksi import ConDB

import sql_connection
import json
import time
import random
import datetime
import copy

import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from matplotlib.patches import FancyArrowPatch

import warnings
warnings.filterwarnings('ignore')

def generate_data(n = 30, random_state = None):
    random.seed(random_state)
    query = """SELECT 
                    p.post_id,
                    p.post_type,
                    CASE
                        WHEN pj.pj_jam_buka = pj.pj_jam_tutup THEN "tutup"
                        ELSE "buka"
                    END AS is_operate
                FROM 
                    posts p
                LEFT JOIN
                    posts_jadwal pj
                    ON p.post_id = pj.pj_id_tempat AND pj.pj_hari = "minggu"
                """

    df_location = sql_connection.read_from_sql(query)
    
    tourid = df_location[(df_location['post_type']=="location")&
                         (df_location['is_operate'] != "tutup")]['post_id'].values.tolist()
    tourid = random.sample(tourid,n)
    idhotel = df_location[df_location['post_type']=="hotel"]['post_id'].values.tolist()
    idhotel = idhotel[random.randint(0,len(idhotel)-1)]
    
    db = ConDB()

    hotel = db.HotelbyID(idhotel)
    tur = db.WisatabyID(tourid)
    timematrix = db.TimeMatrixbyID(hotel._id,tourid)
    
    return hotel,tur,timematrix

# setting
hotel,tur,timematrix = generate_data(n=30,random_state=30) #random state 50 blacklist
travel_days = 3
dwaktu,dtarif,drating = 1,1,1

In [116]:
ga_vrp = GA_VRP(random_state=100)
ga_vrp.set_model(tour=tur,hotel=hotel,timematrix=timematrix,
                 travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
a,b = ga_vrp.construct_solution()

In [117]:
a

[{'index': [73, 76, 93, 9, 86, 18, 10, 4],
  'waktu': [datetime.time(8, 0),
   datetime.time(8, 24),
   datetime.time(9, 32),
   datetime.time(11, 8),
   datetime.time(13, 50),
   datetime.time(14, 24),
   datetime.time(15, 31),
   datetime.time(16, 37),
   datetime.time(17, 43),
   datetime.time(19, 29)],
  'rating': [4.8, 4.5, 4.6, 4.7, 4.4, 4.5, 4.6, 4.6],
  'tarif': [0, 0, 2500, 0, 0, 0, 0, 0]},
 {'index': [39, 34, 71, 88, 46],
  'waktu': [datetime.time(8, 0),
   datetime.time(8, 58),
   datetime.time(11, 8),
   datetime.time(14, 1),
   datetime.time(16, 42),
   datetime.time(17, 52),
   datetime.time(19, 30)],
  'rating': [4.6, 4.5, 4.5, 4.8, 5.0],
  'tarif': [6000, 0, 5000, 0, 0]},
 {'index': [16, 35, 7, 54, 22, 95, 1],
  'waktu': [datetime.time(8, 0),
   datetime.time(8, 20),
   datetime.time(11, 0),
   datetime.time(12, 9),
   datetime.time(13, 47),
   datetime.time(15, 23),
   datetime.time(17, 9),
   datetime.time(18, 14),
   datetime.time(19, 52)],
  'rating': [4.4, 4.6, 4.8

In [118]:
b

0.6560393530631624

In [64]:
results

([{'index': [53, 46, 1, 7, 9, 73],
   'waktu': [datetime.time(8, 0),
    datetime.time(12, 0),
    datetime.time(14, 2),
    datetime.time(15, 36),
    datetime.time(17, 9),
    datetime.time(18, 10),
    datetime.time(18, 57),
    datetime.time(20, 4)],
   'rating': [5.0, 5.0, 4.8, 4.8, 4.7, 4.8],
   'tarif': [25000, 0, 0, 0, 0, 0]},
  {'index': [93, 74, 64, 10, 4],
   'waktu': [datetime.time(8, 0),
    datetime.time(8, 50),
    datetime.time(11, 44),
    datetime.time(14, 18),
    datetime.time(16, 33),
    datetime.time(17, 39),
    datetime.time(19, 25)],
   'rating': [4.6, 4.8, 4.7, 4.6, 4.6],
   'tarif': [2500, 0, 25000, 0, 0]},
  {'index': [39, 69, 35, 88, 71, 22],
   'waktu': [datetime.time(8, 0),
    datetime.time(8, 58),
    datetime.time(10, 55),
    datetime.time(12, 4),
    datetime.time(13, 7),
    datetime.time(14, 48),
    datetime.time(17, 36),
    datetime.time(19, 11)],
   'rating': [4.6, 4.6, 4.6, 4.8, 4.5, 4.5],
   'tarif': [6000, 8000, 10000, 0, 5000, 0]}],
 0.608

In [61]:
ts_vrp.rest_nodes

[Pasar Ngasem(09:32:00-10:17:00),
 Situs Gua Seluman(00:00:00-00:00:00),
 Pakualaman Palace(00:00:00-00:00:00),
 The Lost World Castle(11:11:00-12:41:00),
 Grojogan Watu Purbo Gate(14:43:00-16:13:00),
 Jogja T-Shirt Jethe(16:47:00-17:32:00),
 Paintball Jupiter Jogja(08:21:00-10:21:00),
 Desa Wisata Pentingsari(12:49:00-14:49:00),
 Bukit Pangol(00:00:00-00:00:00),
 Gembira Loka Zoo(13:48:00-15:18:00),
 Klenteng Poncowinatan(00:00:00-00:00:00),
 Mandira Baruga(10:24:00-11:24:00),
 Kampung Wisata Basen(10:32:00-11:17:00)]

In [11]:
tur

[MALIOBORO JOGJAKARTA(00:00:00-00:00:00),
 Bringin Kembar Yogyakarta(00:00:00-00:00:00),
 Titik Nol Km Yogyakarta(00:00:00-00:00:00),
 Tarunasura Wijilan Gate(00:00:00-00:00:00),
 Yogyakarta Northern Square(00:00:00-00:00:00),
 Pasar Ngasem(00:00:00-00:00:00),
 Situs Gua Seluman(00:00:00-00:00:00),
 Pakualaman Palace(00:00:00-00:00:00),
 Bentara Budaya Yogyakarta (BBY)(00:00:00-00:00:00),
 The Lost World Castle(00:00:00-00:00:00),
 Grojogan Watu Purbo Gate(00:00:00-00:00:00),
 House of Chocolate Monggo & Gelato Tirtodipuran(00:00:00-00:00:00),
 Sermo Reservoir Dam(00:00:00-00:00:00),
 Jogja T-Shirt Jethe(00:00:00-00:00:00),
 ALUN ALUN KIDUL YOGYAKARTA(00:00:00-00:00:00),
 Paintball Jupiter Jogja(00:00:00-00:00:00),
 Desa Wisata Pentingsari(00:00:00-00:00:00),
 GOLDEN BIOSKOP VIRTUAL REALITY(00:00:00-00:00:00),
 Bukit Pangol(00:00:00-00:00:00),
 Bukit Rhema Gereja Ayam(00:00:00-00:00:00),
 The Palace of Yogyakarta(00:00:00-00:00:00),
 Hutan Pinus Pengger(00:00:00-00:00:00),
 Gembira Lok

In [1]:
sum([[1,2,3],[4,5,6]],[])

[1, 2, 3, 4, 5, 6]

In [107]:
import random
import copy
random.seed(100)
def partially_mapped_crossover(parent1,parent2): #PMX
    start,end = sorted(random.sample(range(len(parent1)), 2))
    child1 = [None] * len(parent1)
    child2 = copy.deepcopy(child1)
    mapping1 = {parent1[i]._id:parent2[i] for i in range(start,end+1)}
    mapping2 = {parent2[i]._id:parent1[i] for i in range(start,end+1)}
    print(mapping1)
    print(mapping2)

    for i in range(len(parent1)):
        if start <= i <= end:
            child1[i],child2[i] = parent1[i],parent2[i] 
        else:
            gene1,gene2 = parent2[i],parent1[i]
            while gene1._id in mapping1:
                gene1 = mapping1[gene1._id]            
            print(gene2._id)
            while gene2._id in mapping2:
                print("got ", gene2._id)
                gene2 = mapping2[gene2._id]
            child1[i],child2[i] = gene1,gene2

    return child1,child2

partially_mapped_crossover(a,b)
# partially_mapped_crossover(tur[:5],tur[5:10])

{11: Sermo Reservoir Dam(00:00:00-00:00:00), 72: ALUN ALUN KIDUL YOGYAKARTA(00:00:00-00:00:00), 39: GOLDEN BIOSKOP VIRTUAL REALITY(00:00:00-00:00:00), 35: Bentara Budaya Yogyakarta (BBY)(00:00:00-00:00:00), 40: Klenteng Poncowinatan(00:00:00-00:00:00), 51: Pasar Ngasem(00:00:00-00:00:00), 9: Tarunasura Wijilan Gate(00:00:00-00:00:00), 53: Kampung Wisata Basen(00:00:00-00:00:00), 64: MALIOBORO JOGJAKARTA(00:00:00-00:00:00), 7: Hutan Pinus Pengger(00:00:00-00:00:00), 88: Yogyakarta Northern Square(00:00:00-00:00:00)}
{39: Pasar Ngasem(00:00:00-00:00:00), 46: Gembira Loka Zoo(00:00:00-00:00:00), 53: Sermo Reservoir Dam(00:00:00-00:00:00), 22: House of Chocolate Monggo & Gelato Tirtodipuran(00:00:00-00:00:00), 76: Jogja T-Shirt Jethe(00:00:00-00:00:00), 11: Desa Wisata Pentingsari(00:00:00-00:00:00), 9: Tarunasura Wijilan Gate(00:00:00-00:00:00), 95: GOLDEN BIOSKOP VIRTUAL REALITY(00:00:00-00:00:00), 1: Bukit Rhema Gereja Ayam(00:00:00-00:00:00), 71: Titik Nol Km Yogyakarta(00:00:00-00:00:

([The Palace of Yogyakarta(00:00:00-00:00:00),
  Klenteng Poncowinatan(00:00:00-00:00:00),
  Mandira Baruga(00:00:00-00:00:00),
  Situs Gua Seluman(00:00:00-00:00:00),
  Pasar Ngasem(00:00:00-00:00:00),
  Gembira Loka Zoo(00:00:00-00:00:00),
  Sermo Reservoir Dam(00:00:00-00:00:00),
  House of Chocolate Monggo & Gelato Tirtodipuran(00:00:00-00:00:00),
  Jogja T-Shirt Jethe(00:00:00-00:00:00),
  Desa Wisata Pentingsari(00:00:00-00:00:00),
  Tarunasura Wijilan Gate(00:00:00-00:00:00),
  GOLDEN BIOSKOP VIRTUAL REALITY(00:00:00-00:00:00),
  Bukit Rhema Gereja Ayam(00:00:00-00:00:00),
  Titik Nol Km Yogyakarta(00:00:00-00:00:00),
  Jogokariyan Mosque(00:00:00-00:00:00),
  Kampung Wisata Basen(00:00:00-00:00:00),
  Seribu Batu Songgo Langit(00:00:00-00:00:00),
  The Lost World Castle(00:00:00-00:00:00),
  Hutan Pinus Pengger(00:00:00-00:00:00),
  MALIOBORO JOGJAKARTA(00:00:00-00:00:00),
  Bentara Budaya Yogyakarta (BBY)(00:00:00-00:00:00),
  Makam Panembahan Senopati(00:00:00-00:00:00),
  Gr

In [104]:
def partially_mapped_crossover(parent1, parent2):
    # Perform crossover
    start, end = sorted(random.sample(range(len(parent1)), 2))
    child = [None] * len(parent1)
    mapping = {parent1[i]: parent2[i] for i in range(start, end + 1)}

    # Resolve conflicts and complete mapping
    for i in range(len(parent1)):
        if start <= i <= end:
            child[i] = parent1[i]
        else:
            gene = parent2[i]
            while gene in mapping:
                print(gene)
                gene = mapping[gene]
            child[i] = gene

    return child

partially_mapped_crossover([1,2,3,4,5,6],[6,5,4,3,2,1])

5
4


[6, 2, 3, 4, 5, 1]

In [78]:
b

[The Palace of Yogyakarta(00:00:00-00:00:00),
 Jogja T-Shirt Jethe(00:00:00-00:00:00),
 Mandira Baruga(00:00:00-00:00:00),
 Situs Gua Seluman(00:00:00-00:00:00),
 Sermo Reservoir Dam(00:00:00-00:00:00),
 ALUN ALUN KIDUL YOGYAKARTA(00:00:00-00:00:00),
 GOLDEN BIOSKOP VIRTUAL REALITY(00:00:00-00:00:00),
 Bentara Budaya Yogyakarta (BBY)(00:00:00-00:00:00),
 Klenteng Poncowinatan(00:00:00-00:00:00),
 Pasar Ngasem(00:00:00-00:00:00),
 Tarunasura Wijilan Gate(00:00:00-00:00:00),
 Kampung Wisata Basen(00:00:00-00:00:00),
 MALIOBORO JOGJAKARTA(00:00:00-00:00:00),
 Hutan Pinus Pengger(00:00:00-00:00:00),
 Yogyakarta Northern Square(00:00:00-00:00:00),
 Desa Wisata Pentingsari(00:00:00-00:00:00),
 Seribu Batu Songgo Langit(00:00:00-00:00:00),
 The Lost World Castle(00:00:00-00:00:00),
 Titik Nol Km Yogyakarta(00:00:00-00:00:00),
 Bukit Rhema Gereja Ayam(00:00:00-00:00:00),
 House of Chocolate Monggo & Gelato Tirtodipuran(00:00:00-00:00:00),
 Makam Panembahan Senopati(00:00:00-00:00:00),
 Grojoga

In [24]:
# Roulette Wheel Selection for Elimination
def roulette_wheel_selection(population, fitness_values):
    total_fitness = sum(fitness_values)
    roulette_spin = random.uniform(0, total_fitness)
    cumulative_fitness = 0
    for i, fitness in enumerate(fitness_values):
        cumulative_fitness += fitness
        if cumulative_fitness >= roulette_spin:
            return population[i]

In [28]:
parents = [roulette_wheel_selection([[1,2,3,4,5,6],[6,5,4,3,2,1],[7,6,5,34,7,8]], [0.7,0.4,0.9]) for _ in range(3)]
parents

[[6, 5, 4, 3, 2, 1], [7, 6, 5, 34, 7, 8], [1, 2, 3, 4, 5, 6]]

In [29]:
for i in range(0, 3, 2):
    print(i)

0
2


In [38]:
random.choices([[1,2,3,4,5,6],[6,5,4,3,2,1],[7,6,5,34,7,8]],weights=np.array([0.7,0.4,0.9]),k=2)

[[7, 6, 5, 34, 7, 8], [7, 6, 5, 34, 7, 8]]

In [37]:
import numpy as np